# Open-cell tests

In [11]:
%load_ext autoreload
%autoreload 2

# og python libs
import re
import os
import pprint
import itertools
from typing import Tuple, List, Dict, Pattern, Any


# classic DS libs
import pandas as pd
import numpy as np

# viz libs
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.font_manager as font_manager
from ipywidgets import interact, fixed, interact_manual
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

# scipy
from scipy.integrate import quad
from scipy.signal import find_peaks, peak_widths, find_peaks_cwt
from scipy.constants import k as k_B
from scipy.constants import c, pi, N_A, h, hbar, m_e, m_p

# lmfit things
from lmfit.models import LinearModel, GaussianModel, VoigtModel, SplineModel, ExponentialModel, ExpressionModel, Model, ConstantModel
from lmfit import Parameter, Parameters
from lmfit import minimize, minimizer

# ML
from sklearn.linear_model import LinearRegression

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# custom
import sys
sys.path.append('../../../../')
from my_utils import defs as cp
from my_utils.classes import CEPAS_measurement, CEPAS_benchmark, CEPAS_noise_info, CEPAS_SNR_bench
cp.hello_test()

Hello!


In [ ]:
p1 = "./bypass_0/"
p2 = "./bypass_5/"
p3 = "./bypass_5_short/"
p4 = "./bypass_1000/"

p1_items = cp.dir_match(p1, "spectrum_oc")

dfs_dict = {item[:-4]: pd.read_csv(p1 + item, sep="\\s+", header=None) for item in p1_items}



,0,1
0,0.000000,0.050406
1,0.147685,0.071356
2,0.295370,0.071463
3,0.443054,0.071365
4,0.590739,0.070813
